In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [2]:
# Loading data
file_path = "/Users/ythun/OneDrive/Desktop/UNC-VIRT-DATA/Housing-Capstone/Resources/final_housing_df.csv"
df_housing = pd.read_csv(file_path)
df_housing

,Unnamed: 0,lotAreaValue,address,bedrooms,longitude,latitude,zpid,bathrooms,livingArea,CONDO,LOT,SINGLE_FAMILY,TOWNHOUSE,price
0,89,0.179982,27613,3.0,-78.750850,35.925007,6515009,2.5,2007.0,0,0,1,0,490000
1,773,0.260000,27613,3.0,-78.740234,35.924540,6540974,3.0,1991.0,0,0,1,0,460000
2,165,0.920000,27613,4.0,-78.699820,35.925026,80208142,6.0,4978.0,0,0,1,0,1275000
3,195,1.769995,27587,3.0,-78.400520,35.934620,336860835,3.0,2108.0,0,1,0,0,781000
4,881,3.950000,27539,5.0,-78.817566,35.698390,6536910,5.0,4437.0,0,0,1,0,980000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,367,0.920000,27513,4.0,-78.840750,35.787712,6543490,4.0,3446.0,0,0,1,0,1250000
891,531,0.230000,27545,4.0,-78.513245,35.788654,6522355,3.0,2203.0,0,0,1,0,465000
892,892,0.040000,27612,2.0,-78.734880,35.875683,94722912,3.0,1695.0,0,0,0,1,368000
893,189,0.150000,27587,3.0,-78.481540,35.988950,248755438,4.0,2853.0,0,0,1,0,590000


In [3]:
df_housing = df_housing.drop(['Unnamed: 0', 'zpid'], axis=1)
df_housing

,lotAreaValue,address,bedrooms,longitude,latitude,bathrooms,livingArea,CONDO,LOT,SINGLE_FAMILY,TOWNHOUSE,price
0,0.179982,27613,3.0,-78.750850,35.925007,2.5,2007.0,0,0,1,0,490000
1,0.260000,27613,3.0,-78.740234,35.924540,3.0,1991.0,0,0,1,0,460000
2,0.920000,27613,4.0,-78.699820,35.925026,6.0,4978.0,0,0,1,0,1275000
3,1.769995,27587,3.0,-78.400520,35.934620,3.0,2108.0,0,1,0,0,781000
4,3.950000,27539,5.0,-78.817566,35.698390,5.0,4437.0,0,0,1,0,980000
...,...,...,...,...,...,...,...,...,...,...,...,...
890,0.920000,27513,4.0,-78.840750,35.787712,4.0,3446.0,0,0,1,0,1250000
891,0.230000,27545,4.0,-78.513245,35.788654,3.0,2203.0,0,0,1,0,465000
892,0.040000,27612,2.0,-78.734880,35.875683,3.0,1695.0,0,0,0,1,368000
893,0.150000,27587,3.0,-78.481540,35.988950,4.0,2853.0,0,0,1,0,590000


In [4]:
# Get the unique values of 'address' column
df_housing.address.unique()

array([27613, 27587, 27539, 27616, 27518, 27526, 27562, 27529, 27540,
       27607, 27606, 27571, 27615, 27617, 27513, 27603, 27591, 27519,
       27592, 27545, 27604, 27523, 27511, 27609, 27610, 27502, 27614,
       27601, 27560, 27612, 27608, 27605, 27597, 27520, 27501, 27596],
      dtype=int64)

In [5]:
df_housing['address'] = df_housing['address'].astype(float)

In [6]:
# creating a new dataframe 27613
df_27613 = df_housing[df_housing['address'] ==27613]
  
print(df_27613)

     lotAreaValue  address  bedrooms  longitude   latitude  bathrooms  \
0        0.179982  27613.0       3.0 -78.750850  35.925007        2.5   
1        0.260000  27613.0       3.0 -78.740234  35.924540        3.0   
2        0.920000  27613.0       4.0 -78.699820  35.925026        6.0   
28       1.110000  27613.0       4.0 -78.708040  35.920420        3.0   
41       0.020000  27613.0       3.0 -78.713140  35.877820        3.0   
61       0.150000  27613.0       3.0 -78.682920  35.907253        3.0   
68       0.700000  27613.0       3.0 -78.684260  35.891624        2.0   
76       0.080000  27613.0       3.0 -78.708300  35.882790        3.0   
107      3.110000  27613.0       3.0 -78.695040  35.986225        3.0   
118      0.050000  27613.0       3.0 -78.734420  35.888424        3.0   
151      0.170000  27613.0       3.0 -78.707800  35.875870        2.0   
176      0.020000  27613.0       3.0 -78.735435  35.885033        3.0   
295      0.150000  27613.0       4.0 -78.730030  35

In [7]:
# creating a new dataframe 27587
df_27587 = df_housing[df_housing['address'] ==27587]
  
print(df_27587)

     lotAreaValue  address  bedrooms  longitude   latitude  bathrooms  \
3        1.769995  27587.0       3.0  -78.40052  35.934620        3.0   
8        0.180000  27587.0       5.0  -78.51681  35.957382        5.0   
15       0.130000  27587.0       4.0  -78.51207  35.956295        4.0   
16       2.600000  27587.0       3.0  -78.47181  35.883595        3.0   
20      37.950000  27587.0       3.0  -78.49854  36.002277        3.0   
..            ...      ...       ...        ...        ...        ...   
790      1.170000  27587.0       4.0  -78.52806  35.993706        4.0   
792      0.210000  27587.0       4.0  -78.48701  35.941480        4.0   
851      0.360000  27587.0       4.0  -78.51402  35.928640        3.0   
873      1.300000  27587.0       3.0  -78.70187  35.993435        3.0   
893      0.150000  27587.0       3.0  -78.48154  35.988950        4.0   

     livingArea  CONDO  LOT  SINGLE_FAMILY  TOWNHOUSE    price  
3        2108.0      0    1              0          0   78

In [8]:
# creating a new dataframe 27539
df_27539 = df_housing[df_housing['address'] ==27539]
  
print(df_27539)

     lotAreaValue  address  bedrooms  longitude   latitude  bathrooms  \
4        3.950000  27539.0       5.0 -78.817566  35.698390        5.0   
82       0.073000  27539.0       3.0 -78.807450  35.670400        3.0   
99       3.260000  27539.0       5.0 -78.715210  35.688026        6.0   
125      0.070000  27539.0       4.0 -78.825760  35.687008        4.0   
163      0.280000  27539.0       5.0 -78.727040  35.645264        4.0   
192      0.160000  27539.0       4.0 -78.827930  35.683710        3.0   
195      1.100000  27539.0       3.0 -78.737206  35.656326        2.0   
206      0.070000  27539.0       3.0 -78.807170  35.670372        3.0   
308      0.230000  27539.0       4.0 -78.810780  35.676580        3.0   
330      0.290000  27539.0       4.0 -78.818500  35.674004        3.0   
349      1.290000  27539.0       3.0 -78.741560  35.662148        3.0   
420      0.320000  27539.0       4.0 -78.732380  35.643642        3.0   
456      0.280000  27539.0       4.0 -78.759210  35

In [9]:
# creating a new dataframe '27616', '27518', '27526', '27562', '27529', '27540', '27607', '27606', '27571', '27615', '27617',
# '27513', '27603', '27591', '27519', '27592', '27545', '27604', '27523', '27511', '27609', '27610', '27502', '27614', '27601',
# '27560', '27612', '27608', '27605', '27597', '27520', '27501', '27596'
df_27616 = df_housing[df_housing['address'] ==27616]
df_27518 = df_housing[df_housing['address'] ==27518]
df_27526 = df_housing[df_housing['address'] ==27526]
df_27562 = df_housing[df_housing['address'] ==27562]
df_27529 = df_housing[df_housing['address'] ==27529]
df_27540 = df_housing[df_housing['address'] ==27540]
df_27607 = df_housing[df_housing['address'] ==27607]
df_27606 = df_housing[df_housing['address'] ==27606]
df_27571 = df_housing[df_housing['address'] ==27571]
df_27615 = df_housing[df_housing['address'] ==27615]
df_27617 = df_housing[df_housing['address'] ==27617]
df_27513 = df_housing[df_housing['address'] ==27513]
df_27603 = df_housing[df_housing['address'] ==27603]
df_27591 = df_housing[df_housing['address'] ==27591]
df_27519 = df_housing[df_housing['address'] ==27519]
df_27592 = df_housing[df_housing['address'] ==27592]
df_27545 = df_housing[df_housing['address'] ==27545]
df_27604 = df_housing[df_housing['address'] ==27604]
df_27523 = df_housing[df_housing['address'] ==27523]
df_27511 = df_housing[df_housing['address'] ==27511]
df_27609 = df_housing[df_housing['address'] ==27609]
df_27610 = df_housing[df_housing['address'] ==27610]
df_27502 = df_housing[df_housing['address'] ==27502]
df_27614 = df_housing[df_housing['address'] ==27614]
df_27601 = df_housing[df_housing['address'] ==27601]
df_27560 = df_housing[df_housing['address'] ==27560]
df_27612 = df_housing[df_housing['address'] ==27612]
df_27608 = df_housing[df_housing['address'] ==27608]
df_27605 = df_housing[df_housing['address'] ==27605]
df_27597 = df_housing[df_housing['address'] ==27597]
df_27520 = df_housing[df_housing['address'] ==27520]
df_27501 = df_housing[df_housing['address'] ==27501]
df_27596 = df_housing[df_housing['address'] ==27596]

In [10]:
zipcode_df = [df_27616, df_27518, df_27526, df_27562, df_27529, df_27540, df_27607, df_27606, df_27571, df_27615,
              df_27617, df_27513, df_27603, df_27591, df_27519, df_27592, df_27545, df_27604, df_27523, df_27511,
              df_27609, df_27610, df_27502, df_27614, df_27601, df_27560, df_27612, df_27608, df_27605, df_27597,
              df_27520, df_27501, df_27596, df_27539, df_27587, df_27613]
print(zipcode_df)

[     lotAreaValue  address  bedrooms  longitude   latitude  bathrooms  \
5        0.140000  27616.0       3.0 -78.567810  35.855083        3.0   
35       0.169995  27616.0       3.0 -78.566315  35.852386        2.0   
44       0.390000  27616.0       4.0 -78.566160  35.839344        3.0   
54       0.130000  27616.0       4.0 -78.517110  35.867130        3.0   
63       2.819995  27616.0       3.0 -78.586610  35.869200        0.0   
85       0.030000  27616.0       2.0 -78.591210  35.867085        3.0   
114      0.050000  27616.0       3.0 -78.526850  35.849674        3.0   
115      0.280000  27616.0       3.0 -78.517800  35.884212        3.0   
124      0.130000  27616.0       3.0 -78.558440  35.886684        2.0   
146      0.220000  27616.0       4.0 -78.570564  35.855680        3.0   
152      0.150000  27616.0       3.0 -78.568170  35.834370        2.0   
154      0.150000  27616.0       3.0 -78.546776  35.887524        3.0   
169      0.250000  27616.0       5.0 -78.542010  3

In [11]:
df_housing['address'] = df_housing['address'].astype(str)

In [12]:
# Seperate each propertyType and assign a 1 or 0
pd.get_dummies(df_housing['address']).head()

,27501.0,27502.0,27511.0,27513.0,27518.0,27519.0,27520.0,27523.0,27526.0,27529.0,...,27607.0,27608.0,27609.0,27610.0,27612.0,27613.0,27614.0,27615.0,27616.0,27617.0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Drop propertyType column
df_housing.drop('address', axis=1).head()

,lotAreaValue,bedrooms,longitude,latitude,bathrooms,livingArea,CONDO,LOT,SINGLE_FAMILY,TOWNHOUSE,price
0,0.179982,3.0,-78.750850,35.925007,2.5,2007.0,0,0,1,0,490000
1,0.260000,3.0,-78.740234,35.924540,3.0,1991.0,0,0,1,0,460000
2,0.920000,4.0,-78.699820,35.925026,6.0,4978.0,0,0,1,0,1275000
3,1.769995,3.0,-78.400520,35.934620,3.0,2108.0,0,1,0,0,781000
4,3.950000,5.0,-78.817566,35.698390,5.0,4437.0,0,0,1,0,980000


In [14]:
# Join shuffled and dummies dataframes
df_encoded = pd.concat([df_housing.drop('address', axis=1),
                             pd.get_dummies(df_housing['address'])], axis=1)
df_encoded

,lotAreaValue,bedrooms,longitude,latitude,bathrooms,livingArea,CONDO,LOT,SINGLE_FAMILY,TOWNHOUSE,...,27607.0,27608.0,27609.0,27610.0,27612.0,27613.0,27614.0,27615.0,27616.0,27617.0
0,0.179982,3.0,-78.750850,35.925007,2.5,2007.0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1,0.260000,3.0,-78.740234,35.924540,3.0,1991.0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,0.920000,4.0,-78.699820,35.925026,6.0,4978.0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,1.769995,3.0,-78.400520,35.934620,3.0,2108.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3.950000,5.0,-78.817566,35.698390,5.0,4437.0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,0.920000,4.0,-78.840750,35.787712,4.0,3446.0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
891,0.230000,4.0,-78.513245,35.788654,3.0,2203.0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
892,0.040000,2.0,-78.734880,35.875683,3.0,1695.0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
893,0.150000,3.0,-78.481540,35.988950,4.0,2853.0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df_encoded.to_csv('df encoded.csv')